
Step 1. [Download history price and clean data](/notebooks/forex/hawkeye/1_download.ipynb)

Step 2. [Use monte carlo simulate each stock's movement for 10000 times](/notebooks/forex/hawkeye/2_process.ipynb)

**Step 3. Apply some heuristic rules to give stock recommendation**

In [18]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import sys

sys.path.append('/opt/hawkeye')
from asx import *

this_week=20190524
last_week=20190517
path = f'/opt/hawkeye/data/{this_week}/result.csv'
last_path = f'/opt/hawkeye/data/{last_week}/result.csv'

def week_return_avg(df):
    return round(df['week_return'].dropna().sum()/len(df['week_return'].dropna()),4)

df = pd.read_csv(path,
                 usecols=[
                     'code', 'last_date', 'start price', 'sim_mean',
                     'sim_diff', 'VaR 99%', 'VaR 99% Percent', 'volume_mean',
                     'return_mean', 'return_sigma'
                 ],
                 index_col='code')
last_df = pd.read_csv(last_path,
                      usecols=[
                          'code', 'last_date', 'start price', 'sim_mean',
                          'sim_diff', 'VaR 99%', 'VaR 99% Percent',
                          'volume_mean', 'return_mean', 'return_sigma'
                      ],
                      index_col='code')

print(f'Total stock number = {len(df)}')

# remove bottom volume and bottom price
volume_threshold = df['volume_mean'].quantile(0.1)
df = df[(df['volume_mean'] > volume_threshold) & (df['start price'] > 0.05)]

# calculate return & risk rank
df['return'] = round(df['sim_diff'] / df['start price'] * 100, 3)
df['return_rank'] = round(df['return'].rank(pct=True) * 100, 3)
df['risk_rank'] = round(df['VaR 99% Percent'].rank(pct=True) * 100, 3)
df['volume_rank'] = round(df['volume_mean'].rank(pct=True) * 100, 3)

# last week performance
df['last_return'] = round(last_df['sim_diff'] / last_df['start price'] * 100,3)
df['return_increase'] = df['return'] - df['last_return']
df['week_return'] = round(
    (df['start price'] - last_df['start price']) / df['start price'] * 100, 3)

# drop used columns
df.drop(columns=['sim_diff', 'sim_mean', 'VaR 99%','VaR 99% Percent', 'volume_mean'], inplace=True)

print(f'Total stock number after bottom volume & price removed = {len(df)}')
print('Price 70%% pricetile = %s' % df['start price'].quantile(0.7))
print('Sim return 90%% percentile = %s' % df['return'].quantile(0.9))

df.sort_values(by='return', ascending=False).head(20)

Total stock number = 940
Total stock number after bottom volume & price removed = 725
Price 70% pricetile = 2.411999999999998
Sim return 90% percentile = 1.1268000000000005


,last_date,start price,return_mean,return_sigma,return,return_rank,risk_rank,volume_rank,last_return,return_increase,week_return
code,,,,,,,,,,,
ANO,2019-05-27,7.020,0.022919,0.066730,4.705,100.000,77.931,10.483,3.639,1.066,33.048
AVH,2019-05-27,0.455,0.020646,0.075820,3.976,99.862,88.828,98.345,4.350,-0.374,-7.692
LTR,2019-05-28,0.110,0.018294,0.092349,3.816,99.724,96.000,98.759,2.795,1.021,35.455
8CO,2019-05-27,0.155,0.017755,0.075269,3.636,99.586,90.483,35.862,3.297,0.339,0.000
WZR,2019-05-28,0.170,0.018665,0.100016,3.627,99.448,98.759,94.897,3.359,0.268,17.647
FEX,2019-05-28,0.094,0.017862,0.091920,3.483,99.310,96.966,98.207,2.660,0.823,17.021
ATU,2019-05-27,0.340,0.016598,0.075959,3.341,99.172,89.793,63.724,3.331,0.010,-2.941
ISX,2019-05-27,0.660,0.016576,0.056584,3.180,99.034,70.345,82.897,1.653,1.527,49.242
LBT,2019-05-28,0.205,0.015325,0.151159,3.003,98.897,100.000,70.483,0.011,2.992,60.000


In [2]:
# high mean return 
df.sort_values(by='return', ascending=False).head(10)

,last_date,start price,return_mean,return_sigma,return,return_rank,risk_rank,volume_rank,last_return,return_increase,week_return
code,,,,,,,,,,,
ANO,2019-05-27,7.020,0.022919,0.066730,4.705,100.000,77.931,10.483,3.639,1.066,33.048
AVH,2019-05-27,0.455,0.020646,0.075820,3.976,99.862,88.828,98.345,4.350,-0.374,-7.692
LTR,2019-05-28,0.110,0.018294,0.092349,3.816,99.724,96.000,98.759,2.795,1.021,35.455
8CO,2019-05-27,0.155,0.017755,0.075269,3.636,99.586,90.483,35.862,3.297,0.339,0.000
WZR,2019-05-28,0.170,0.018665,0.100016,3.627,99.448,98.759,94.897,3.359,0.268,17.647
FEX,2019-05-28,0.094,0.017862,0.091920,3.483,99.310,96.966,98.207,2.660,0.823,17.021
ATU,2019-05-27,0.340,0.016598,0.075959,3.341,99.172,89.793,63.724,3.331,0.010,-2.941
ISX,2019-05-27,0.660,0.016576,0.056584,3.180,99.034,70.345,82.897,1.653,1.527,49.242
LBT,2019-05-28,0.205,0.015325,0.151159,3.003,98.897,100.000,70.483,0.011,2.992,60.000


In [3]:
# high price
df[df['start price']>1].sort_values(by='return', ascending=False).head(10)

,last_date,start price,return_mean,return_sigma,return,return_rank,risk_rank,volume_rank,last_return,return_increase,week_return
code,,,,,,,,,,,
ANO,2019-05-27,7.020,0.022919,0.066730,4.705,100.000,77.931,10.483,3.639,1.066,33.048
5GN,2019-05-27,1.350,0.012917,0.046811,2.579,98.483,58.759,43.448,2.371,0.208,9.630
Z1P,2019-05-28,3.460,0.012759,0.042866,2.412,98.207,54.759,85.655,2.547,-0.135,-1.734
BUB,2019-05-27,1.265,0.011732,0.053853,2.362,98.069,71.448,94.759,2.220,0.142,0.791
CIA,2019-05-27,2.900,0.010158,0.039037,1.979,96.690,50.759,36.000,2.007,-0.028,3.103
JIN,2019-05-27,17.130,0.009134,0.027798,1.802,96.138,28.828,43.034,1.883,-0.081,-2.977
MGX,2019-05-27,1.245,0.009362,0.028459,1.773,96.000,28.966,88.690,1.950,-0.177,-0.803
NEA,2019-05-27,3.440,0.008758,0.033279,1.752,95.586,44.000,88.000,1.991,-0.239,-11.337
CRD,2019-05-27,1.660,0.008834,0.043928,1.750,95.379,59.724,37.517,1.861,-0.111,-3.916


In [4]:
# highest price
df[df['return']>0.8].sort_values(by='start price', ascending=False).head(10)

,last_date,start price,return_mean,return_sigma,return,return_rank,risk_rank,volume_rank,last_return,return_increase,week_return
code,,,,,,,,,,,
MFG,2019-05-27,43.90,0.006793,0.019041,1.330,92.276,12.276,59.172,1.392,-0.062,-2.096
CUV,2019-05-27,33.68,0.006901,0.033153,1.339,92.552,46.897,13.931,1.211,0.128,6.591
APX,2019-05-27,26.24,0.007777,0.032044,1.559,94.483,40.966,67.034,1.614,-0.055,4.002
APT,2019-05-27,23.93,0.007714,0.038569,1.505,93.379,51.310,83.172,1.696,-0.191,-6.979
BRG,2019-05-27,17.28,0.005489,0.023411,1.061,88.138,25.655,41.793,1.158,-0.097,-5.382
JIN,2019-05-27,17.13,0.009134,0.027798,1.802,96.138,28.828,43.034,1.883,-0.081,-2.977
IEL,2019-05-27,16.59,0.005701,0.025645,1.111,89.517,31.448,64.138,1.163,-0.052,-0.784
IVC,2019-05-27,15.97,0.004838,0.020115,0.948,85.103,19.862,56.828,0.884,0.064,3.193
WEB,2019-05-28,15.15,0.004333,0.037057,0.831,81.724,55.862,59.310,0.951,-0.120,-10.891


In [19]:
# low risk
low_risk = df.sort_values(by='risk_rank', ascending=True).head(20).sort_values(by='return', ascending=False)
print('Low risk last week performance avg. = %s%%'% week_return_avg(low_risk))
low_risk

Low risk last week performance avg. = 0.155%


,last_date,start price,return_mean,return_sigma,return,return_rank,risk_rank,volume_rank,last_return,return_increase,week_return
code,,,,,,,,,,,
CHC,2019-05-27,10.570,0.003729,0.010259,0.709,77.379,1.241,80.276,0.665,0.044,0.662
MGR,2019-05-27,3.120,0.003588,0.010326,0.690,76.414,1.379,99.448,0.510,0.180,5.449
ASX,2019-05-27,76.420,0.002721,0.008721,0.526,68.000,0.966,45.379,0.561,-0.035,0.406
AMC,2019-05-27,16.160,0.002333,0.007838,0.454,63.103,0.690,91.724,0.532,-0.078,-2.723
DXS,2019-05-27,13.160,0.002309,0.009577,0.453,62.828,2.069,91.172,0.356,0.097,0.912
BXB,2019-05-27,12.190,0.002195,0.009808,0.444,62.621,2.345,91.448,0.453,-0.009,-3.035
CMW,2019-05-27,1.190,0.002109,0.007670,0.421,60.483,0.138,92.552,0.392,0.029,0.420
SHL,2019-05-28,26.260,0.002152,0.008966,0.394,57.931,1.655,73.241,0.424,-0.030,-0.724
AST,2019-05-27,1.790,0.001817,0.007741,0.365,55.931,0.828,92.966,0.327,0.038,-3.911


In [7]:
# high volume
df.sort_values(by='volume_rank', ascending=False).head(20)

,last_date,start price,return_mean,return_sigma,return,return_rank,risk_rank,volume_rank,last_return,return_increase,week_return
code,,,,,,,,,,,
TLS,2019-05-28,3.540,0.002428,0.012408,0.476,64.414,6.207,100.000,0.439,0.037,0.847
FMG,2019-05-27,8.450,0.008591,0.027545,1.673,95.034,29.793,99.862,1.646,0.027,-5.917
S32,2019-05-28,3.475,0.000969,0.014498,0.215,44.759,13.379,99.724,0.293,-0.078,-1.295
HSO,2019-05-27,2.460,0.001342,0.006879,0.253,48.414,0.414,99.586,0.380,-0.127,0.407
MGR,2019-05-27,3.120,0.003588,0.010326,0.690,76.414,1.379,99.448,0.510,0.180,5.449
AWC,2019-05-27,2.430,0.001813,0.017312,0.364,55.724,19.310,99.310,0.334,0.030,1.646
VCX,2019-05-28,2.700,0.000752,0.010502,0.156,40.276,5.655,99.172,-0.044,0.200,4.444
SCG,2019-05-27,3.920,0.000541,0.010721,0.084,33.793,6.069,99.034,-0.034,0.118,2.041
AMP,2019-05-27,2.180,-0.000726,0.021988,-0.148,18.897,36.966,98.897,-0.064,-0.084,-0.917


In [8]:
# highest market cap.
highest_market_cap = [
    'BHP', 'RIO', 'CBA', 'WBC', 'CSL', 'ANZ', 'NAB', 'MQG', 'WOW', 'WES',
    'TLS', 'WPL'
    'S32', 'FMG', 'BXB', 'NCM', 'IAG', 'SUN', 'AMC', 'QBE', 'CIM', 'ALL',
    'AGL', 'ASX', 'RHC', 'OSH', 'SHL', 'TWE', 'REA', 'COH'
]
asx_20 = get_asx_20_list()
df[df.index.isin(asx_20)].sort_values(by='return', ascending=False).head(20)

,last_date,start price,return_mean,return_sigma,return,return_rank,risk_rank,volume_rank,last_return,return_increase,week_return
code,,,,,,,,,,,
RIO,2019-05-27,103.020,0.003350,0.013688,0.648,74.207,8.000,80.690,0.647,0.001,1.621
TLS,2019-05-28,3.540,0.002428,0.012408,0.476,64.414,6.207,100.000,0.439,0.037,0.847
AMC,2019-05-27,16.160,0.002333,0.007838,0.454,63.103,0.690,91.724,0.532,-0.078,-2.723
BXB,2019-05-27,12.190,0.002195,0.009808,0.444,62.621,2.345,91.448,0.453,-0.009,-3.035
WES,2019-05-28,38.150,0.002222,0.012288,0.430,61.517,7.724,86.345,0.335,0.095,4.692
ANZ,2019-05-27,27.820,0.001955,0.014615,0.394,57.931,13.931,95.034,0.242,0.152,7.081
WPL,2019-05-28,36.005,0.001754,0.011270,0.350,54.759,5.931,85.379,0.520,-0.170,-3.319
WBC,2019-05-28,27.920,0.001572,0.015265,0.319,52.552,14.207,96.966,0.195,0.124,8.990
IAG,2019-05-27,7.890,0.001560,0.010892,0.301,51.448,5.379,94.345,0.342,-0.041,-1.774


## Thisk week recommendations

In [15]:
# 2019-05-17 recommendations
selection = [
    'AVH',  # high return & high risk
    'ANO',  # high return & high risk
    'AGH',  # high return & meduim risk
    'Z1P',  # medium price & risk
    'MFG',  # high price & low risk
    'AD8',  # low risk
    'FMG',  # high volume & low risk
]
last_week_df = df[df.index.isin(selection)]
last_week_df.sort_values(by='return', ascending=False).head(10)

,last_date,start price,return_mean,return_sigma,return,return_rank,risk_rank,volume_rank,last_return,return_increase,week_return
code,,,,,,,,,,,
ANO,2019-05-27,7.020,0.022919,0.066730,4.705,100.000,77.931,10.483,3.639,1.066,33.048
AVH,2019-05-27,0.455,0.020646,0.075820,3.976,99.862,88.828,98.345,4.350,-0.374,-7.692
Z1P,2019-05-28,3.460,0.012759,0.042866,2.412,98.207,54.759,85.655,2.547,-0.135,-1.734
AGH,2019-05-27,0.660,0.010286,0.045925,2.032,97.103,60.828,53.379,2.718,-0.686,6.061
FMG,2019-05-27,8.450,0.008591,0.027545,1.673,95.034,29.793,99.862,1.646,0.027,-5.917
AD8,2019-05-27,7.120,0.007307,0.024062,1.440,92.966,23.862,27.862,1.438,0.002,0.140
MFG,2019-05-27,43.900,0.006793,0.019041,1.330,92.276,12.276,59.172,1.392,-0.062,-2.096


In [16]:
week_return = round(sum(last_week_df['week_return'])/len(selection), 3)
win_rate = round(len(last_week_df[last_week_df['week_return'] >= 0]) / len(selection), 3)
print('Last week average return:', week_return, ' , win rate:', win_rate)

Last week average return: 3.116  , win rate: 0.429


## Last week review

In [14]:
selection = [
    'AVH',  # high return & high risk
    'ANO',  # high return & high risk
    'AGH',  # high return & meduim risk
    'Z1P',  # medium price & risk
    'MFG',  # high price & low risk
    'AD8',  # low risk
    'FMG',  # high volume & low risk
]
df[df.index.isin(selection)].sort_values(by='return', ascending=False).head(10)

,last_date,start price,return_mean,return_sigma,return,return_rank,risk_rank,volume_rank,last_return,return_increase,week_return
code,,,,,,,,,,,
ANO,2019-05-27,7.020,0.022919,0.066730,4.705,100.000,77.931,10.483,3.639,1.066,33.048
AVH,2019-05-27,0.455,0.020646,0.075820,3.976,99.862,88.828,98.345,4.350,-0.374,-7.692
Z1P,2019-05-28,3.460,0.012759,0.042866,2.412,98.207,54.759,85.655,2.547,-0.135,-1.734
AGH,2019-05-27,0.660,0.010286,0.045925,2.032,97.103,60.828,53.379,2.718,-0.686,6.061
FMG,2019-05-27,8.450,0.008591,0.027545,1.673,95.034,29.793,99.862,1.646,0.027,-5.917
AD8,2019-05-27,7.120,0.007307,0.024062,1.440,92.966,23.862,27.862,1.438,0.002,0.140
MFG,2019-05-27,43.900,0.006793,0.019041,1.330,92.276,12.276,59.172,1.392,-0.062,-2.096


## Signle performance review

In [54]:
df[df.index=='FEX']

,last_date,start price,VaR 99% Percent,volume_mean,return_mean,return_sigma,return,last_return,return_increase,week_return
code,,,,,,,,,,


# Return & Risk Rank

In [17]:
df['rank'] = df['return_rank'] + (100 - df['risk_rank'])
top_df = df.sort_values(by='rank', ascending=False).head(20)
week_return = round(sum(top_df['week_return'].dropna())/len(top_df), 3)
print('Top rank last week average return:', week_return)
top_df

Top rank last week average return: 1.551


,last_date,start price,return_mean,return_sigma,return,return_rank,risk_rank,volume_rank,last_return,return_increase,week_return,rank
code,,,,,,,,,,,,
MFG,2019-05-27,43.900,0.006793,0.019041,1.330,92.276,12.276,59.172,1.392,-0.062,-2.096,180.000
CHC,2019-05-27,10.570,0.003729,0.010259,0.709,77.379,1.241,80.276,0.665,0.044,0.662,176.138
MGR,2019-05-27,3.120,0.003588,0.010326,0.690,76.414,1.379,99.448,0.510,0.180,5.449,175.035
DDR,2019-05-27,5.120,0.006555,0.021161,1.264,91.310,18.069,20.690,1.281,-0.017,-0.391,173.241
AD8,2019-05-27,7.120,0.007307,0.024062,1.440,92.966,23.862,27.862,1.438,0.002,0.140,169.104
MND,2019-05-27,19.430,0.003830,0.015268,0.736,78.759,11.034,46.621,0.780,-0.044,2.367,167.725
JIN,2019-05-27,17.130,0.009134,0.027798,1.802,96.138,28.828,43.034,1.883,-0.081,-2.977,167.310
MGX,2019-05-27,1.245,0.009362,0.028459,1.773,96.000,28.966,88.690,1.950,-0.177,-0.803,167.034
ASX,2019-05-27,76.420,0.002721,0.008721,0.526,68.000,0.966,45.379,0.561,-0.035,0.406,167.034
